In [1]:
import os
import os.path as osp
import pandas as pd
import datetime
import json
from typing import List
import math

In [2]:
heart_rate_folder = '/home/nvtu/PhD_Work/StressDetection/MyDataset/Fitbit/HeartRate'
gsr_signal_folder = '/home/nvtu/PhD_Work/StressDetection/MyDataset/GSR'

In [3]:
def ceiling_time(timestamp: str):
    _time = datetime.datetime.strptime(timestamp, '%H:%M:%S:%f')
    _time_delta = datetime.timedelta(hours = _time.hour, minutes = _time.minute, seconds = _time.second, microseconds = _time.microsecond)
    total_secs = math.ceil(_time_delta.total_seconds())
    _time = datetime.datetime.utcfromtimestamp(total_secs).strftime('%H:%M:%S')
    return _time

In [4]:
def matching(hr_data, timestamps):
    data = []
    _time = [item[0] for item in hr_data]
    for timestamp in timestamps:
        try:
            index = _time.index(timestamp)
            hr_value = hr_data[index][1]
            data.append((timestamp, hr_value))
        except:
            data.append((timestamp, None))
    return data

In [5]:
gsr_file_names = sorted(os.listdir(gsr_signal_folder), reverse = True)
for file_name in gsr_file_names:
    file_path = osp.join(gsr_signal_folder, file_name)
    file_name = osp.splitext(file_name)[0]
    # Get the date from the file name and re-format it
    date = file_name.split()[-3]
    date = datetime.datetime.strptime(date, '%d.%m.%y').date().__str__()
    # Get targeted heart file path
    targeted_hr_file_path = osp.join(heart_rate_folder, date + '.json')
    # Load heart rate and date time information from signal data
    hr_data = json.load(open(targeted_hr_file_path, 'r'))['activities-heart-intraday']['dataset']
    hr_data = [(item['time'], item['value']) for item in hr_data]
    timestamps = pd.read_csv(file_path)['TIMESTAMP'].tolist()
    timestamps = list(map(ceiling_time, timestamps))
    matched_data = matching(hr_data, timestamps)
    print(matched_data) 
    break

ValueError: '16:27:49' is not in list